In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from fastai.vision import *
from fastai import * 
from fastai.text import *
from pathlib import Path 
from tqdm import tqdm
tqdm.pandas(desc="my bar!")

In [3]:
import torchtext
from torchtext.data import Iterator, BucketIterator
from torchvision import transforms, utils

In [4]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)
def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

# Language Model

In [5]:
lm = True
clean = False

In [6]:
if lm: path = Path('../input'); 
list(path.iterdir())

[WindowsPath('../input/embeddings'),
 WindowsPath('../input/glove.840B.300d'),
 WindowsPath('../input/sample_submission.csv'),
 WindowsPath('../input/test.csv'),
 WindowsPath('../input/tmp_lm'),
 WindowsPath('../input/train.csv')]

In [7]:
if clean:
    train_df["question_text"] = train_df["question_text"].apply(lambda x: x.lower())
    test_df["question_text"] = test_df["question_text"].apply(lambda x: x.lower())
    allText_df["question_text"] = allText_df["question_text"].apply(lambda x: x.lower())

    train_df['question_text'] = train_df['question_text'].apply(lambda x: clean_text(x))
    test_df['question_text'] = test_df['question_text'].apply(lambda x: clean_text(x))
    allText_df["question_text"] = allText_df["question_text"].apply(lambda x: clean_text(x))

    train_df['question_text'] = train_df['question_text'].apply(lambda x: clean_numbers(x))
    test_df['question_text'] = test_df['question_text'].apply(lambda x:clean_numbers(x))
    allText_df["question_text"] = allText_df["question_text"].apply(lambda x: clean_numbers(x))

    train_df['question_text'] = train_df['question_text'].apply(lambda x: replace_typical_mispell(x))
    test_df['question_text'] = test_df['question_text'].apply(lambda x: replace_typical_mispell(x))
    allText_df["question_text"] = allText_df["question_text"].apply(lambda x: replace_typical_misspell(x))

#train_x = train_df['question_text'].fillna('_##_').values
#test_x = test_df['question_text'].fillna('_##_').values

import zipfile
zf = zipfile.ZipFile(path/'test.csv.zip')
df_test = pd.read_csv(zf.open('test.csv'))
zf = zipfile.ZipFile(path/'train.csv.zip')
df_train = pd.read_csv(zf.open('train.csv'))



In [8]:
#print(df_train.head(3))

In [9]:
# df_train.head(3)

In [10]:
def prepare_sequences():
    tokenizer = lambda text: text.split() # the function above is the function we will be using to tokenize the text
    TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer, lower=True)
    LABEL = torchtext.data.Field(sequential=False, use_vocab=False) # sequential and use_vocab=False since no text (binary)
    train_datafields = [("qid", None), ("question_text", TEXT), ("target", LABEL)]
    train = torchtext.data.TabularDataset( # If we had a validation set as well, we would add an additional .splits(...)
                        # path="data/train_cleaned_v2.csv", # the root directory where the data lies
                        path = path/'train.csv',
                        format='csv',
                        # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
                        skip_header=True, 
                        fields=train_datafields)
    test_datafields = [("qid", None),
                     ("question_text", TEXT), ("target", LABEL)] 
    test = torchtext.data.TabularDataset( 
                path=path/'test.csv',
                format="csv",
                skip_header=True,
                fields=test_datafields)
    return TEXT, LABEL, train, test
# Chat Conversation End
# Type a message...



In [11]:
TEXT,LABEL,train,test = prepare_sequences()


In [12]:
LABEL.batch_first


False

In [13]:
TEXT.build_vocab(train, test, vectors = "glove.6B.50d")

In [14]:
train,_ = train.split(0.1)

In [15]:
from torchtext.data import Iterator, BucketIterator
train_iter = BucketIterator(
    train, # we pass in the datasets we want the iterator to draw data from
    batch_size=256, 
    sort_key= lambda x: len(x.question_text), # the BucketIterator needs to be told what function it should use to group the data.
    sort_within_batch=False, # sorting would add bias
    repeat=False)
test_iter = Iterator(test, batch_size = 256, device = torch.device(0), sort = False,  sort_within_batch = False, repeat = False) #Changed device from -1 to None


In [16]:
#iter(train_iter).__next__().question_text
len(train_iter)
#train_iter = iter(train_iter)
#train_iter.__next__().question_text
#train_iter.__next__().target
#iter(train_iter).__next__().question_text.shape

511

In [17]:
#smaller,_ = train.split(0.1)

In [18]:
#smaller.examples[0].question_text

In [19]:
#len(train)

In [20]:
#train.examples[0].question_text

In [21]:
TEXT.vocab.vectors.shape

torch.Size([527626, 50])

In [22]:
TEXT.vocab.freqs

Counter({'how': 369907,
         'did': 52902,
         'quebec': 122,
         'nationalists': 132,
         'see': 11665,
         'their': 44817,
         'province': 174,
         'as': 77136,
         'a': 523949,
         'nation': 1073,
         'in': 480479,
         'the': 853487,
         '1960s?': 68,
         'do': 312432,
         'you': 241782,
         'have': 116324,
         'an': 86247,
         'adopted': 403,
         'dog,': 106,
         'would': 78740,
         'encourage': 476,
         'people': 63109,
         'to': 521322,
         'adopt': 574,
         'and': 329249,
         'not': 54759,
         'shop?': 244,
         'why': 200208,
         'does': 110586,
         'velocity': 821,
         'affect': 5640,
         'time?': 7418,
         'space': 3121,
         'geometry?': 111,
         'otto': 62,
         'von': 145,
         'guericke': 3,
         'used': 15907,
         'magdeburg': 2,
         'hemispheres?': 6,
         'can': 227414,
         

In [23]:
embed_size = 50

In [24]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet,self).__init__()
        
        hidden_size = 64
        
        self.embedding = nn.Embedding(len(TEXT.vocab),embed_size)
        self.embedding.weight = nn.Parameter(TEXT.vocab.vectors)
        self.embedding_dropout = nn.Dropout2d(0.1)
        
        self.lstm = nn.LSTM(embed_size, hidden_size, bidirectional=True, batch_first=True)
        self.gru = nn.GRU(hidden_size*2, hidden_size, bidirectional=True, batch_first=True)
        
        self.linear = nn.Linear(hidden_size*4,2)
        self.relu = nn.ReLU()
        #self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(16,1)
        
    def forward(self,train):
        #print(train.shape)
        h_embedding = self.embedding(train)
        #print(h_embedding.shape)
        h_embedding = torch.squeeze(self.embedding_dropout(torch.unsqueeze(h_embedding,0)))
        #print(h_embedding.shape)
        
        h_lstm, _ = self.lstm(h_embedding)
        #print(h_lstm.shape)
        h_gru, _ = self.gru(h_lstm)
        #print(h_gru.shape)
        
        avg_pool = torch.mean(h_gru,1)
        #print(avg_pool.shape)
        #avg_pool = avg_pool.view(-1,64)
        max_pool, _ = torch.max(h_gru,1)
        #print(max_pool.shape)
        
        
        conc = torch.cat((avg_pool, max_pool),1)
        print(conc.shape)
        conc = self.relu(self.linear(conc))
        #print(conc.shape)
        #conc = self.dropout(conc)
        print(conc.shape)
        #out = nn.Sigmoid()(conc)
        
        out = nn.LogSoftmax(dim=1)(conc)
        
        #out = self.out(conc)
        #print(out[:,-1].shape)
        
        return out
    
    
    
    
batch = next(iter(train_iter)).question_text
net = NeuralNet()
#net(batch)

In [25]:
#batch = next(iter(train_iter)).question_text
#batch[30]

In [26]:
class LSTMTagger(nn.Module):
    def __init__(self):
        super(LSTMTagger, self).__init__()
        
        hidden_size = 6
        
        self.hidden_dim = hidden_size
        
        self.embedding = nn.Embedding(len(TEXT.vocab),embed_size)
        self.embedding.weight = nn.Parameter(TEXT.vocab.vectors)
        #self.embedding_dropout = nn.Dropout2d(0.1)
        
        self.lstm = nn.LSTM(embed_size, hidden_size)
        
        self.hidden2tag = nn.Linear(hidden_size, 2)
    
    def forward(self, train):
        #print(train.shape)
        embeds = self.embedding(train)
        #print(embeds.shape)
        #print(embeds.view(len(train),64,-1).shape)
        #print(embeds.view(len(train),train.shape[1],-1).shape)
        lstm_out, _ = self.lstm(embeds.view(len(train),train.shape[1],-1))
        #print(lstm_out.shape)
        avg_pool = torch.mean(lstm_out,0)
        #print(avg_pool.shape)
        target_space = self.hidden2tag(avg_pool)#.view(len(train),-1))
        #print(target_space.shape)
        target_score = F.log_softmax(target_space, dim=1)
        #print(target_score.shape)
        return target_score

inputs_check = next(iter(train_iter)).question_text

#print(inputs_check.shape)

model = LSTMTagger()

with torch.no_grad():
    inputs = inputs_check
    #print(len(inputs))
    score = model(inputs)
    #print(score)

In [28]:
model = LSTMTagger()
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(),lr=0.1)

inputs_check = next(iter(train_iter)).question_text

with torch.no_grad():
    inputs = inputs_check
    score = model(inputs)
    print(score)
    
for epoch in range(1):
    for batch_num in range(len(train_iter)):
        model.zero_grad()
        print(batch_num,"out of: ",len(train_iter))
        sentence_data = next(iter(train_iter))
        sentence_in = sentence_data.question_text
        targets = sentence_data.target
        
        target_scores = model(sentence_in)
        
        loss = loss_function(target_scores, targets)
        print(loss)
        loss.backward()
        optimizer.step()

with torch.no_grad():
    inputs = inputs_check
    score = model(inputs)
    print(score)
        
        

tensor([[-0.9397, -0.4955],
        [-0.9360, -0.4979],
        [-0.9256, -0.5047],
        [-0.9498, -0.4891],
        [-0.9271, -0.5037],
        [-0.9559, -0.4852],
        [-0.9399, -0.4954],
        [-0.9254, -0.5048],
        [-0.9422, -0.4939],
        [-0.9159, -0.5111],
        [-0.9479, -0.4903],
        [-0.8890, -0.5294],
        [-0.9237, -0.5059],
        [-0.9445, -0.4924],
        [-0.9189, -0.5091],
        [-0.9355, -0.4982],
        [-0.9516, -0.4880],
        [-0.9352, -0.4984],
        [-0.9625, -0.4812],
        [-0.9124, -0.5134],
        [-0.9520, -0.4877],
        [-0.8975, -0.5235],
        [-0.9156, -0.5113],
        [-0.9147, -0.5119],
        [-0.9469, -0.4909],
        [-0.9699, -0.4766],
        [-0.9212, -0.5076],
        [-0.9472, -0.4907],
        [-0.9660, -0.4790],
        [-0.9336, -0.4994],
        [-0.9413, -0.4945],
        [-0.9315, -0.5008],
        [-0.9138, -0.5125],
        [-0.9319, -0.5005],
        [-0.9033, -0.5196],
        [-0.9257, -0

tensor(0.2500, grad_fn=<NllLossBackward>)
18 out of:  511
tensor(0.2792, grad_fn=<NllLossBackward>)
19 out of:  511
tensor(0.2333, grad_fn=<NllLossBackward>)
20 out of:  511
tensor(0.2833, grad_fn=<NllLossBackward>)
21 out of:  511
tensor(0.2357, grad_fn=<NllLossBackward>)
22 out of:  511
tensor(0.2512, grad_fn=<NllLossBackward>)
23 out of:  511
tensor(0.2514, grad_fn=<NllLossBackward>)
24 out of:  511
tensor(0.2143, grad_fn=<NllLossBackward>)
25 out of:  511
tensor(0.1777, grad_fn=<NllLossBackward>)
26 out of:  511
tensor(0.2948, grad_fn=<NllLossBackward>)
27 out of:  511
tensor(0.2190, grad_fn=<NllLossBackward>)
28 out of:  511
tensor(0.2172, grad_fn=<NllLossBackward>)
29 out of:  511
tensor(0.2576, grad_fn=<NllLossBackward>)
30 out of:  511
tensor(0.2067, grad_fn=<NllLossBackward>)
31 out of:  511
tensor(0.2557, grad_fn=<NllLossBackward>)
32 out of:  511
tensor(0.2066, grad_fn=<NllLossBackward>)
33 out of:  511
tensor(0.2638, grad_fn=<NllLossBackward>)
34 out of:  511
tensor(0.2879,

tensor(0.3043, grad_fn=<NllLossBackward>)
158 out of:  511
tensor(0.2726, grad_fn=<NllLossBackward>)
159 out of:  511
tensor(0.2314, grad_fn=<NllLossBackward>)
160 out of:  511
tensor(0.2978, grad_fn=<NllLossBackward>)
161 out of:  511
tensor(0.2995, grad_fn=<NllLossBackward>)
162 out of:  511
tensor(0.2164, grad_fn=<NllLossBackward>)
163 out of:  511
tensor(0.1856, grad_fn=<NllLossBackward>)
164 out of:  511
tensor(0.2376, grad_fn=<NllLossBackward>)
165 out of:  511
tensor(0.2376, grad_fn=<NllLossBackward>)
166 out of:  511
tensor(0.2286, grad_fn=<NllLossBackward>)
167 out of:  511
tensor(0.2198, grad_fn=<NllLossBackward>)
168 out of:  511
tensor(0.2119, grad_fn=<NllLossBackward>)
169 out of:  511
tensor(0.1839, grad_fn=<NllLossBackward>)
170 out of:  511
tensor(0.2057, grad_fn=<NllLossBackward>)
171 out of:  511
tensor(0.1813, grad_fn=<NllLossBackward>)
172 out of:  511
tensor(0.1358, grad_fn=<NllLossBackward>)
173 out of:  511
tensor(0.1448, grad_fn=<NllLossBackward>)
174 out of:  5

296 out of:  511
tensor(0.1890, grad_fn=<NllLossBackward>)
297 out of:  511
tensor(0.1876, grad_fn=<NllLossBackward>)
298 out of:  511
tensor(0.1802, grad_fn=<NllLossBackward>)
299 out of:  511
tensor(0.2628, grad_fn=<NllLossBackward>)
300 out of:  511
tensor(0.2812, grad_fn=<NllLossBackward>)
301 out of:  511
tensor(0.1917, grad_fn=<NllLossBackward>)
302 out of:  511
tensor(0.2179, grad_fn=<NllLossBackward>)
303 out of:  511
tensor(0.2008, grad_fn=<NllLossBackward>)
304 out of:  511
tensor(0.1381, grad_fn=<NllLossBackward>)
305 out of:  511
tensor(0.2469, grad_fn=<NllLossBackward>)
306 out of:  511
tensor(0.2669, grad_fn=<NllLossBackward>)
307 out of:  511
tensor(0.2762, grad_fn=<NllLossBackward>)
308 out of:  511
tensor(0.2045, grad_fn=<NllLossBackward>)
309 out of:  511
tensor(0.2261, grad_fn=<NllLossBackward>)
310 out of:  511
tensor(0.2067, grad_fn=<NllLossBackward>)
311 out of:  511
tensor(0.1218, grad_fn=<NllLossBackward>)
312 out of:  511
tensor(0.2744, grad_fn=<NllLossBackward

tensor(0.1592, grad_fn=<NllLossBackward>)
435 out of:  511
tensor(0.2400, grad_fn=<NllLossBackward>)
436 out of:  511
tensor(0.2492, grad_fn=<NllLossBackward>)
437 out of:  511
tensor(0.1939, grad_fn=<NllLossBackward>)
438 out of:  511
tensor(0.1970, grad_fn=<NllLossBackward>)
439 out of:  511
tensor(0.1447, grad_fn=<NllLossBackward>)
440 out of:  511
tensor(0.2820, grad_fn=<NllLossBackward>)
441 out of:  511
tensor(0.2559, grad_fn=<NllLossBackward>)
442 out of:  511
tensor(0.2698, grad_fn=<NllLossBackward>)
443 out of:  511
tensor(0.1422, grad_fn=<NllLossBackward>)
444 out of:  511
tensor(0.2314, grad_fn=<NllLossBackward>)
445 out of:  511
tensor(0.2012, grad_fn=<NllLossBackward>)
446 out of:  511
tensor(0.2550, grad_fn=<NllLossBackward>)
447 out of:  511
tensor(0.2062, grad_fn=<NllLossBackward>)
448 out of:  511
tensor(0.1758, grad_fn=<NllLossBackward>)
449 out of:  511
tensor(0.2009, grad_fn=<NllLossBackward>)
450 out of:  511
tensor(0.1929, grad_fn=<NllLossBackward>)
451 out of:  5

In [29]:
print(score)

tensor([[-0.0486, -3.0477],
        [-0.0643, -2.7756],
        [-0.0509, -3.0025],
        [-0.0917, -2.4350],
        [-0.0505, -3.0100],
        [-0.0553, -2.9225],
        [-0.0592, -2.8556],
        [-0.0450, -3.1234],
        [-0.0662, -2.7486],
        [-0.0491, -3.0388],
        [-0.0738, -2.6431],
        [-0.0730, -2.6534],
        [-0.0652, -2.7622],
        [-0.0556, -2.9177],
        [-0.0448, -3.1275],
        [-0.0469, -3.0836],
        [-0.0517, -2.9875],
        [-0.0522, -2.9787],
        [-0.0509, -3.0042],
        [-0.0437, -3.1512],
        [-0.0584, -2.8697],
        [-0.0621, -2.8094],
        [-0.0460, -3.1028],
        [-0.0485, -3.0512],
        [-0.0606, -2.8339],
        [-0.0868, -2.4873],
        [-0.0442, -3.1400],
        [-0.0589, -2.8615],
        [-0.0723, -2.6631],
        [-0.0550, -2.9279],
        [-0.0577, -2.8808],
        [-0.0473, -3.0745],
        [-0.0747, -2.6318],
        [-0.0467, -3.0869],
        [-0.0580, -2.8754],
        [-0.0471, -3

In [ ]:
train_iters = iter(train_iter)

In [ ]:
wee = next(train_iters)

In [ ]:
wee.question_text.shape